### Segmenting and Clustering neighborhoods in Toronto

In [110]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/postal-codes-1/Postal_codes(1).txt
/kaggle/input/postal/Postal_codes_TO(1).txt
/kaggle/input/toronto-postal-codes/codes_text(2).txt


#### Web-scraping using Beautiful Soup package. 


Importing required dependencies

In [111]:
import requests
import lxml.html as lh
import pandas as pd
from bs4 import BeautifulSoup

defining the url

In [112]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

initiating response request and making sure that response is fulfilled. A status_code of 200 means that the response was correctly given

In [113]:
response = requests.get(url)
response.status_code

200

Storing the web-page content and finding the table

In [114]:
response.content
soup = BeautifulSoup(response.content, 'html.parser')
codes_table = soup.find_all('table' , class_ = 'wikitable sortable')
codes_table = codes_table[0]

Extracting each row and content of each row into a dataframe

In [115]:
list_of_rows = []
for row in codes_table.findAll('tr'):
    list_of_cells = []
    for cell in row.findAll('td'):
        text = cell.text
        list_of_cells.append(text)
    list_of_rows.append(list_of_cells)

row_vals = list_of_rows

In [116]:
df = pd.DataFrame(columns = ['a','b','c'])
df = df.append(row_vals)

The above dataframe needs to be cleaned and handled correctly

In [117]:
df['a'] = df[0]
df['b'] = df[1]
df['c'] = df[2]

df.drop(columns=[0,1,2],inplace=True)

#Dropping rows with no Borough data

df = df[df.b != 'Not assigned\n']
df = df.iloc[1:]

df['a'] = df['a'].astype('string')
df['b'] = df['b'].astype('string')
df['c'] = df['c'].astype('string')

df['a'] = df['a'].str[:3]
df['b'] = df['b'].str[:-1]
df['c'] = df['c'].str[:-1]

df.reset_index(inplace=True)
df.drop(columns=['index'],inplace=True)
df.rename(columns={"a": "Postal Code", "b": "Borough","c" : "Neighborhood"},inplace = True)

#Grouping similar Postal Codes

df['Neighborhood'] = df['Neighborhood'].str.replace("/",", ")

df

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill N..."
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business reply mail Processing Centre
101,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea..."


In [118]:
df.shape

(103, 3)

The above dataframe follows all the guidelines and has 103 rows and 3 columns